In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
job=pd.read_excel("job_info_split.xlsx")
job=job.drop("Unnamed: 0",axis=1)
course=pd.read_excel("course_info_split.xlsx")
course=course.drop("Unnamed: 0",axis=1)

In [3]:
import random
idx=random.sample(range(len(job)), 100000 )
job2=pd.DataFrame(job.values[idx], columns=job.columns)
print("job隨機採計數：", len(job2))
#job2.head()

print("===確認隨機狀況==============================")
print("===不同年份資料數：")
print("2015:", len(job2[job2["yyyymm"]==201501]))
print("2016:", len(job2[job2["yyyymm"]==201601]))
print("2017:", len(job2[job2["yyyymm"]==201701]))
print("2018:", len(job2[job2["yyyymm"]==201801]))
print("===不同職類資料數：(職類/隨機取樣數/取樣數與元資料數比值)")
for i in list(set(job["jobcat"])):
    print(i,":",len(job2[job2["jobcat"]==i]), "||", round(len(job2[job2["jobcat"]==i])/len(job[job["jobcat"]==i]),3))
    
job2=job2[["index","斷詞_feature"]]
course2=course[["index","斷詞_feature"]]

feature_df=pd.concat([job2,course2], axis=0, ignore_index=True)
#feature_df.to_excel("feature_index_split.xlsx")

job隨機採計數： 100000
===確認隨機狀況==============================
===不同年份資料數：
2015: 25133
2016: 25989
2017: 26869
2018: 22009
===不同職類資料數：(職類/隨機取樣數/取樣數與元資料數比值)
營建施作類人員 : 1420 || 0.307
營建規劃類人員 : 1444 || 0.299
化工材料研發類人員 : 375 || 0.294
經營∕幕僚類人員 : 1669 || 0.309
貿易類人員 : 1099 || 0.304
製圖∕測量類人員 : 3028 || 0.31
醫療保健服務人員 : 514 || 0.319
軍警消防類人員 : 138 || 0.275
文字編譯類人員 : 598 || 0.315
金融專業相關類人員 : 3076 || 0.303
生產管理類人員 : 1410 || 0.309
美容∕美髮類人員 : 840 || 0.303
人力資源類人員 : 1595 || 0.313
財務∕會計∕稅務類 : 4661 || 0.307
MIS∕網管類人員 : 2155 || 0.306
農林漁牧相關類人員 : 56 || 0.354
教育輔導類人員 : 1719 || 0.3
工程研發類人員 : 5066 || 0.303
行政∕總務類人員 : 6140 || 0.308
保全類人員 : 347 || 0.302
生技∕醫療研發類人員 : 405 || 0.308
學術研究類人員 : 229 || 0.312
維修∕技術服務類人員 : 2032 || 0.303
傳播藝術類人員 : 403 || 0.305
產品企劃類人員 : 815 || 0.298
記者及採訪類人員 : 145 || 0.331
餐飲類人員 : 2325 || 0.308
專案∕產品管理類人員 : 1238 || 0.307
軟體∕工程類人員 : 7984 || 0.303
門市營業類人員 : 4129 || 0.306
法務∕智財類人員 : 623 || 0.296
運輸物流類人員 : 1016 || 0.313
醫療專業人員 : 4531 || 0.307
客戶服務類人員 : 1943 || 0.312
業務銷售類人員 : 11817 || 0.3
旅遊休閒類人員 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
print(feature_df.shape)

ff=list(feature_df["斷詞_feature"])
vectorizer = TfidfVectorizer()
f_ar = vectorizer.fit_transform(ff)
f_cloumns=vectorizer.get_feature_names()
print(f_ar.shape)

fs=np.sum(f_ar.T, axis=1).tolist()
for i in range(len(fs)):
    fs[i]=fs[i][0]

fsdf=pd.DataFrame({"word":f_cloumns,"count":fs})
fsdf=fsdf.set_index("word", drop=True)
fsdf=fsdf.sort_values("count", ascending=False)

fsdf_s=fsdf.head(5000)
with open("關鍵字B_top_tfidf_5000.txt", "w", encoding="utf-8") as x:
    x.write(" ".join(list(fsdf_s.index)))


df=pd.DataFrame(f_ar.toarray(), columns=f_cloumns, index=feature_df["index"])
df=df[list(fsdf_s.index)]
df.head()

del feature_df
##釋出記憶體
#df.to_csv("feature_tfidf_table.csv") ##看情況看要不要存檔

(108647, 2)
(108647, 15180)


In [5]:
from datetime import datetime

cat=[]
for i in df.index:
    if "j" in i:
        cat.append("j")
    if "c" in i:
        cat.append("c")
df["cate"]=cat

feature_df_j=df[df["cate"]=="j"]
feature_df_j=feature_df_j.reset_index(drop=False)
feature_df_c2=df[df["cate"]=="c"]
cctarget="c38,c39,c40,c199,c200,c201,c8570,c8571,c8572,c8573,c8574,c8575,c8515,c8516,c8517,c8567,c8568,c8569,c245,c246,c247,c248,c249,c250"
cctarget=cctarget.split(",")
feature_df_c=pd.DataFrame()
for ct in cctarget:
    fd_c=feature_df_c2[feature_df_c2.index==ct]
    feature_df_c=pd.concat([feature_df_c, fd_c], axis=0)
feature_df_c=feature_df_c.reset_index(drop=False)

In [6]:
feature_df_c.head()

,index,excel,word,powerpoint,outlook,打字,autocad,50,windows,20,...,防洪,後設,隨行,mapgis,guard,氧氣,tandem,codetest,古典詩,cate
0,c38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
1,c39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
2,c40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
3,c199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
4,c200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c


In [7]:
feature_df_j.head()

,index,excel,word,powerpoint,outlook,打字,autocad,50,windows,20,...,防洪,後設,隨行,mapgis,guard,氧氣,tandem,codetest,古典詩,cate
0,j21260,0.125822,0.128006,0.152473,0.179484,0.461868,0.00000,0.473338,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
1,j243053,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
2,j123473,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
3,j30871,0.000000,0.084044,0.000000,0.117842,0.000000,0.16572,0.000000,0.157833,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
4,j141693,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j


In [8]:
c_index=list(feature_df_c["index"])
j_index=list(feature_df_j["index"])
print("course資料數：", len(c_index))
print("job資料數：", len(j_index))

print("=========================================")
print("total training:", len(feature_df_c))

final_ar=np.zeros((len(feature_df_c), len(feature_df_j)))
for i in range(len(feature_df_c)):
    print(i, "||", datetime.now().strftime("%H:%M:%S"))
    for j in range(len(feature_df_j)):
        cc=np.array(list(feature_df_c.iloc[i,1:-1]))
        cs=np.sum((cc+1)**2)
        jj=np.array(list(feature_df_j.iloc[j,1:-1]))
        js=np.sum((jj+1)**2)
        d=round(np.sum(cc*jj)/np.sqrt(cs)*np.sqrt(js),5)
        #print(d)
        final_ar[i][j]=d

course資料數： 24
job資料數： 100000
total training: 24
0 || 15:02:07
1 || 15:12:32
2 || 15:23:32
3 || 15:34:51
4 || 15:45:52
5 || 15:57:03
6 || 16:10:56
7 || 16:23:22
8 || 16:35:25
9 || 16:46:41
10 || 16:57:31
11 || 17:08:20
12 || 17:19:18


KeyboardInterrupt: 

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

# c_index=list(feature_df_c["index"])
# j_index=list(feature_df_j["index"])
# print("course資料數：", len(c_index))
# print("job資料數：", len(j_index))

# print("=========================================")
# print("total training:", len(feature_df_c))

# final_ar=np.zeros((len(feature_df_c), len(feature_df_j)))
# for i in range(len(feature_df_c)):
#     print(i, "||", datetime.now().strftime("%H:%M:%S"))
#     for j in range(len(feature_df_j)):
#         cc=np.array(list(feature_df_c.iloc[i,1:-1])).reshape(1,5000)
#         jj=np.array(list(feature_df_j.iloc[j,1:-1])).reshape(1,5000)
#         d=cosine_similarity(cc, jj, dense_output=True)
#         #print(d)
#         final_ar[i][j]=d

In [9]:
final_df=pd.DataFrame(final_ar.T, columns=c_index, index=j_index)

In [13]:
final_df2=final_df.iloc[:,0:12]

In [14]:
final_df2.head()

,c38,c39,c40,c199,c200,c201,c8570,c8571,c8572,c8573,c8574,c8575
j21260,0.01036,0.01060,0.00994,0.01079,0.01110,0.00847,0.04106,0.04257,0.03170,0.02117,0.01090,0.00000
j243053,0.07956,0.08140,0.07633,0.08290,0.08523,0.07701,0.04181,0.04335,0.03832,0.03837,0.03952,0.04140
j123473,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
j30871,0.03055,0.03126,0.02931,0.03024,0.03109,0.03331,0.00628,0.00930,0.01069,0.01248,0.01561,0.01539
j141693,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [18]:
final_df.shape

(100000, 12)

In [21]:
jjj=[]
ccc=[]
vvv=[]
for i in range(final_df.shape[0]):
    for j in range(final_df.shape[1]):
        jjj.append(final_df.index[i])
        ccc.append(final_df.columns[j])
        vvv.append(final_df.iloc[i,j])

In [22]:
df=pd.DataFrame({"job":jjj,"course":ccc,"similarity":vvv})

In [23]:
df.to_csv("toptfidf_similarity_table.csv",encoding="utf8")